In [ ]:
import json
import openai
import os
import requests

# Load API keys
openai.api_key = os.environ['OPENAI_API_KEY']
UNIPILE_API_KEY = 'he3d4HCv.b4xXXuTuPelxuS17yRPvmowdX+u0Nua8ihJGiXXinnw='
UNIPILE_BASE_URL = "http://127.0.0.1:3114"

def lambda_handler(event, context):
    try:
        print("Received event:")
        print(json.dumps(event))
        
        # 1. Parse incoming event
        body = json.loads(event['body'])
        message_text = body.get('message', '')
        sender_name = body.get('sender', {}).get('attendee_name', 'there')
        chat_id = body.get('chat_id', None)
        
        if not chat_id:
            return {'statusCode': 400, 'body': json.dumps({'error': 'No chat_id found'})}

        # 2. Prepare prompt for OpenAI
        user_prompt = f"Message from {sender_name}: {message_text}\n\n"
        full_prompt = f"""You are a media assistant from BaseOne. When users ask general questions, politely introduce yourself as 'BaseOne Media Assistant' and tell them to reach out to BaseOne team for specific inquiries. 

If it is a technical or detailed question, answer it fully using your knowledge.

Here is the user's message:
{user_prompt}
"""

        # 3. Generate response from OpenAI
        completion = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant from BaseOne."},
                {"role": "user", "content": full_prompt}
            ],
            max_tokens=300,
            temperature=0.5
        )
        
        openai_response = completion.choices[0].message['content']
        print(f"OpenAI response: {openai_response}")

        # 4. Send the reply to Unipile
        unipile_url = f"{UNIPILE_BASE_URL}/api/v1/chats/{chat_id}/messages"
        headers = {
            "accept": "application/json",
            "X-API-KEY": UNIPILE_API_KEY
        }

        # Send as multipart/form-data
        data = {
            "text": (None, openai_response)
        }

        unipile_response = requests.post(unipile_url, headers=headers, files=data)
        print(f"Unipile response: {unipile_response.status_code} {unipile_response.text}")

        if unipile_response.status_code not in [200, 201]:
            raise Exception(f"Unipile API failed: {unipile_response.text}")

        # 5. Respond back to webhook
        return {
            'statusCode': 200,
            'body': json.dumps({
                'message': 'Reply sent successfully',
                'reply': openai_response,
                'chat_id': chat_id,
                'sender': sender_name
            })
        }

    except Exception as e:
        print(f"Error: {str(e)}")
        return {
            'statusCode': 500,
            'body': json.dumps({'error': str(e)})
        }


TypeError: lambda_handler() missing 2 required positional arguments: 'event' and 'context'